In [1]:
import findspark
findspark.init("/usr/hdp/2.6.5.0-292/spark2")
import pyspark
sc = pyspark.SparkContext()

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
from nltk.corpus import stopwords
import re as re

In [4]:
from pyspark.ml.feature import CountVectorizer , IDF

In [5]:
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel

In [6]:
sqlContext.read.csv('/user/maria_dev/data/newsarticles_article.csv', sep=',', escape='"', header=True, 
               inferSchema=True, multiLine=True).count()

465160

In [7]:
data = sqlContext.read.csv('/user/maria_dev/data/newsarticles_article.csv', sep=',', escape='"', header=False, 
               inferSchema=True, multiLine=True)
tag = sqlContext.read.csv('/user/maria_dev/data/tags.csv', sep=',',  header=True, inferSchema=True, multiLine=True).withColumnRenamed("article_id\r", "article_id")

In [8]:
#data.head()

In [9]:
# tag.columns  

In [10]:
#tag.head()

In [11]:
df = data.join(tag, data._c0 == tag.article_id )

In [12]:
# df.count()

In [13]:
# df =df.na.drop(subset=['category_id','_c4','_c5'])

In [14]:
#df.count()

In [15]:
df2 = df.sample(False, 0.2) 

In [16]:
dfsplit = df2.randomSplit([1.0, 2.0],24)

In [17]:
# dfsplit[0].count()

In [18]:
# dfsplit[1].count()

In [19]:
reviews_train = dfsplit[0].rdd.map(lambda x : str(x['_c4']) + str(x['_c5']))
reviews_test = dfsplit[1].rdd.map(lambda x : str(x['_c4']) + str(x['_c5']))

In [20]:
reviews_train.top(1)

['“Bully Bandit” Suspected In 16th Bank Robbery**CHICAGO (STMW) -** The “Bully Bandit” is suspected of striking again Thursday \nmorning in south suburban Riverside, in his fourth suspected bank robbery in the\n past month after a year-and-a-half hiatus.\n\nHe is suspected of robbing a Bank of America branch at 3300 S. Harlem Ave.\nabout 11:15 a.m., the FBI said. The man, who was wearing a dark or gray hooded\nChicago Bears sweatshirt, threatened to hurt employees and left with an\nunspecified amount of cash, authorities said. He implied he had a weapon but\nnever showed one.\n\nThe man is also suspected of robbing three Chicago banks since Oct. 11. — more\nthan a year and a half after being suspected of a robbery in May 2013, the FBI\nsaid.\n\nThe FBI is offering a reward of up to $17,500 for information that leads to\nthe identification and arrest of the robber.\n\nThe robber is dubbed “The Bully Bandit” because of the aggressive manner in\nwhich he carries out his crimes, the FBI sa

In [21]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
StopWords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maria_dev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/maria_dev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
def tokenlize(doc):
    import nltk, re
    from nltk.corpus import stopwords
    from nltk.corpus import wordnet

    r = re.compile(r'[\w]+')
    my_stopwords = nltk.corpus.stopwords.words('english')
    porter = nltk.PorterStemmer()

    newdoc = []
    for word in nltk.regexp_tokenize(doc, r): 
        newWord = porter.stem(word.lower()) 
        if newWord in my_stopwords: 
            continue
        tokenSynsets = wordnet.synsets(newWord)
        newdoc.append(newWord if tokenSynsets == [] else tokenSynsets[0].lemma_names()[0])
    return newdoc

In [23]:
traintokens = reviews_train.map( lambda doc: tokenlize(doc))
testtokens = reviews_test.map( lambda doc: tokenlize(doc))

In [24]:
from pyspark.mllib.feature import HashingTF
hasingTF = HashingTF(2 ** 16)
trainTf = hasingTF.transform(traintokens)

In [25]:
from pyspark.mllib.feature import IDF
idf = IDF().fit(trainTf)
trainTfidf = idf.transform(trainTf)

In [26]:
from pyspark.mllib.regression import LabeledPoint
trainLabels = dfsplit[0].rdd.map(lambda x : x['category_id'])
train = trainLabels.zip(trainTfidf).map(lambda pair : LabeledPoint(pair[0], pair[1]))

In [27]:
from pyspark.mllib.feature import HashingTF
testTf = hasingTF.transform(testtokens )

In [28]:
testTfidf = idf.transform(testTf)

In [29]:
testLabels = dfsplit[1].rdd.map(lambda x : x['category_id'])
test = testLabels.zip(testTfidf).map(lambda pair : LabeledPoint(pair[0], pair[1]))

In [32]:
testCount= test.count()

In [33]:
from pyspark.mllib.classification import NaiveBayes
model = NaiveBayes.train(train, 0.1)
print('model trained')

model trained


In [34]:
predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda x: x[0] == x[1]).count() / testCount
print(accuracy)

0.843298634582


In [35]:
from  pyspark.mllib.classification import LogisticRegressionWithLBFGS
lrModel = LogisticRegressionWithLBFGS.train(train, iterations=10, numClasses=20) 
print('model trained')

model trained


In [36]:
predictionAndLabel = test.map(lambda p: (lrModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda x: x[0] == x[1]).count() / testCount
print(accuracy )

0.862897120454
